# Resnet 50 Training on (Fake)Imagenet with WebDataset

This notebook illustrates how to use WebDataset with PyTorch training.

In [ ]:
# imports

%matplotlib inline

from functools import partial
from pprint import pprint
import random
from collections import deque
import numpy as np
from matplotlib import pyplot as plt
import sys
import os

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torch.utils.data import DataLoader
from torch import nn, optim

# helpers

import time


def enumerate_report(seq, delta, growth=1.0):
    last = 0
    count = 0
    for count, item in enumerate(seq):
        now = time.time()
        if now - last > delta:
            last = now
            yield count, item, True
        else:
            yield count, item, False
        delta *= growth

In [ ]:
# We usually abbreviate webdataset as wds
import webdataset as wds

In [ ]:
# parameters
epochs = 1
max_steps = int(1e12)
batchsize = 32
bucket = "https://storage.googleapis.com/webdataset/fake-imagenet"
training_urls = bucket + "/imagenet-train-{000000..001281}.tar"

# Data Loader Construction

In [ ]:

# WebDataset is designed to work without any local storage. Use caching
# only if you are on a desktop with slow networking.

if 'google.colab' in sys.modules:
    cache_dir = None
    print("running on colab, streaming data directly from storage")
else:
    !mkdir -p ./_cache
    cache_dir = "./_cache"
    print(f"not running in colab, caching data locally in {cache_dir}")

In [ ]:
# The standard TorchVision transformations.

transform_train = transforms.Compose(
    [
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)


def make_sample(sample, val=False):
    """Take a decoded sample dictionary, augment it, and return an (image, label) tuple."""
    assert not val, "only implemented training dataset for this notebook"
    image = sample["jpg"]
    label = sample["cls"]
    return transform_train(image), label

In [ ]:
# Create the datasets with shard and sample shuffling and decoding.
trainset = wds.WebDataset(
    training_urls, resampled=True, cache_dir=cache_dir, shardshuffle=True
)
trainset = trainset.shuffle(1000).decode("pil").map(make_sample)

# Since this is an IterableDataset, PyTorch requires that we batch in the dataset.
# WebLoader is PyTorch DataLoader with some convenience methods.
trainset = trainset.batched(64)
trainloader = wds.WebLoader(trainset, batch_size=None, num_workers=4)

# Unbatch, shuffle between workers, then rebatch.
trainloader = trainloader.unbatched().shuffle(1000).batched(64)

# Since we are using resampling, the dataset is infinite; set an artificial epoch size.
trainloader = trainloader.with_epoch(1282 * 100 // 64)

In [ ]:
# Smoke test it.

os.environ["GOPEN_VERBOSE"] = "1"
images, classes = next(iter(trainloader))
print(images.shape, classes.shape)
os.environ["GOPEN_VERBOSE"] = "0"

# PyTorch Training

This is a typical PyTorch training pipeline.

In [ ]:
# The usual PyTorch model definition. We use an uninitialized ResNet50 model.

model = resnet50(pretrained=False)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
losses, accuracies = deque(maxlen=100), deque(maxlen=100)

steps = 0

# Train the model
for epoch in range(epochs):
    for i, data, verbose in enumerate_report(trainloader, 5):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = outputs.cpu().detach().argmax(dim=1, keepdim=True)
        correct = pred.eq(labels.cpu().view_as(pred)).sum().item()
        accuracy = correct / float(len(labels))

        losses.append(loss.item())
        accuracies.append(accuracy)
        steps += len(inputs)

        if verbose and len(losses) > 5:
            print(
                "[%d, %5d] loss: %.5f correct: %.5f"
                % (epoch + 1, i + 1, np.mean(losses), np.mean(accuracies))
            )
            running_loss = 0.0

        if steps > max_steps:
            break

    if steps > max_steps:
        break

print("Finished Training")